# Überprüfen der $S_n$ Mengen

Bei der Bestimmung der $S_n$ Mengen wurde die Retrograde Analyse verwendet.
Um sicherzustellen, dass die Bretter die bspw. in $S_{10}$ zu finden sind, auch wirklich in <=10 Zügen beendet werden.

In [ ]:
import json
import chess

In [ ]:
def get_board_and_turn(fen):
    split = fen.split()
    short_fen = str(split[0]) + " " + str(split[1])
    return short_fen

In [ ]:
def load_s_n_sequence(filepath):
    s_n_sequence_fen = []
    s_n_sequence_short_fen = []
    f = open(filepath, "r")
    tmp = json.loads(f.read())
    for item in tmp:
        tmp_fen = set()
        tmp_fen_short = set()
        for board in item:
            tmp_fen.add(board)
            tmp_fen_short.add(get_board_and_turn(board))
        s_n_sequence_fen.append(tmp_fen)
        s_n_sequence_short_fen.append(tmp_fen_short)
    f.close()
    return s_n_sequence_fen, s_n_sequence_short_fen

In [ ]:
S_N_Sequence_fen, S_N_Sequence_fen_short = load_s_n_sequence("S_n_seq_12_01.json")

In [ ]:
def fen_in_lower_sequence(fen, sequence_index, s_n_short):
    for s in s_n_short[:sequence_index]:
        if fen in s:
            return True
    return False

In [ ]:
def every_move_of_sequence_in_lower(sequence_index, s_n_fen, s_n_short):
    print("Checking S_" + str(sequence_index) + "...")
    sequence_fen = s_n_fen[sequence_index]
    for fen in sequence_fen:
        cur_board = chess.Board(fen)
        if not cur_board.turn:
            for move in cur_board.legal_moves:
                cur_board.push(move)
                cur_short = get_board_and_turn(cur_board.fen())
                if not fen_in_lower_sequence(cur_short, sequence_index, s_n_short):
                    print("Fen: " + cur_short + " not in lower S")
                    return False
                cur_board.pop()
            return True
        else:
            in_lower = False
            for move in cur_board.legal_moves:
                cur_board.push(move)
                cur_short = get_board_and_turn(cur_board.fen())
                if fen_in_lower_sequence(cur_short, sequence_index, s_n_short):
                    in_lower = True
                cur_board.pop()
            return False


In [ ]:
for i in range(len(S_N_Sequence_fen)):
    every_move_of_sequence_in_lower(i, S_N_Sequence_fen, S_N_Sequence_fen_short)

Checking S_0...
Checking S_1...
Checking S_2...
Checking S_3...
Checking S_4...
Checking S_5...
Checking S_6...
Checking S_7...
Checking S_8...
Checking S_9...
Checking S_10...
Checking S_11...
Checking S_12...
Checking S_13...
Checking S_14...
Checking S_15...
Checking S_16...
Checking S_17...
Checking S_18...
Checking S_19...
Checking S_20...
Checking S_21...
Checking S_22...
Checking S_23...
Checking S_24...
Checking S_25...
Checking S_26...
Checking S_27...
Checking S_28...
Checking S_29...
Checking S_30...
Checking S_31...
Checking S_32...
Checking S_33...
Checking S_34...


In [ ]:
def find_fen_in_sequence(fen, s_n_short):
    short = get_board_and_turn(fen)
    for i in range(len(s_n_short)):
        if short in s_n_short[i]:
            return i

In [ ]:
find_fen_in_sequence("8/8/8/2K5/k7/6R1/8/8 b - - 0 1", S_N_Sequence_fen_short)

2

In [ ]:
def find_move_count(sequence_index, fen, move_count, move_set, s_n_short):
    cur_board = chess.Board(fen)
    moves = cur_board.legal_moves
    if moves.count() == 0 and sequence_index == 0:
        move_set.add(move_count)
        return move_set
    if not cur_board.turn:
        # print("Black:")
        for move in cur_board.legal_moves:
            cur_board.push(move)
            move_count += 1
            cur_fen = cur_board.fen()
            new_seq = find_fen_in_sequence(cur_fen, s_n_short)
            # print(new_seq, ";" , cur_fen)
            move_set = find_move_count(new_seq, cur_fen, move_count, move_set, s_n_short)
            move_count -= 1
            cur_board.pop()
    else: 
        # print("White:")
        for move in cur_board.legal_moves:
            cur_board.push(move)
            cur_fen = get_board_and_turn(cur_board.fen())
            if fen_in_lower_sequence(cur_fen, sequence_index, s_n_short):
                new_seq = find_fen_in_sequence(cur_fen, s_n_short)
                move_count += 1
                move_set = find_move_count(new_seq, cur_fen, move_count, move_set, s_n_short)
                move_count -= 1
            cur_board.pop()
    return move_set

In [ ]:
find_move_count(20, "8/8/8/3K4/8/5R2/2k5/8 b - - 0 1", 0, set(), S_N_Sequence_fen_short)

{6, 8, 10, 12, 14, 16, 18, 20}

8/8/8/8/2R5/1K1k4/8/8 b - - 0 1


In [ ]:
# for i in range(len(S_N_Sequence_fen)):
invalid = 0
print("Comparing S_" + str(4) + "...")
print( len (S_N_Sequence_fen[4]))
for fen in S_N_Sequence_fen[4]:
    move_set = find_move_count(5, fen, 0, set(), S_N_Sequence_fen_short)
    # print(move_set)
    if max(move_set) != 5:
        invalid += 1
        # print(fen)
print("Invalid boards: " + str(invalid))

Comparing S_4...
1804
Invalid boards: 1804


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=994ae29f-d329-421f-8622-e5bdbf81795f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>